# LAB 2 - Linear Regression Part II
### INFO 251: Applied Machine Learning

* Author: Simón Ramírez Amaya
* Date: Jan 28, 2026

## Helper Functions

#### Execute these cells to helper functions and modules into memory. No need to write any code here.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import FuncFormatter

In [ ]:
def sample_schooling_income(
    n,
    mu_x=14.0,
    mu_y=60000.0,
    sigma_x=2.5,
    sigma_y=15000.0,
    rho=0.7,
    break_1=12.0,
    break_2=16.0,
    jump_1=0,
    jump_2=0,
    random_state=None
):
    """
    Generate a random sample of size n from the schooling-income distribution
    with discontinuities in the conditional mean.

    Parameters
    ----------
    n : int
        Sample size.
    mu_x, mu_y : float
        Means of schooling and baseline income.
    sigma_x, sigma_y : float
        Standard deviations of schooling and income noise.
    rho : float
        Correlation used for slope of conditional mean.
    break_1, break_2 : float
        Schooling levels where income jumps occur.
    jump_1, jump_2 : float
        Magnitude of the jumps at the break points.
    random_state : int or None
        Seed for reproducibility.

    Returns
    -------
    X : ndarray of shape (n,)
        Simulated schooling values.
    Y : ndarray of shape (n,)
        Simulated income values.
    """

    rng = np.random.default_rng(random_state)

    # Step 1: sample schooling
    X = rng.normal(loc=mu_x, scale=sigma_x, size=n)

    # Step 2: compute conditional mean
    slope = rho * (sigma_y / sigma_x)
    m_X = mu_y + slope * (X - mu_x)
    m_X += jump_1 * (X >= break_1)
    m_X += jump_2 * (X >= break_2)

    # Step 3: sample income conditional on schooling
    Y = rng.normal(loc=m_X, scale=sigma_y, size=n)

    return X, Y

# Exercise 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from matplotlib.ticker import FuncFormatter

def ols_repeated_sampling(
    n=500,
    m=10000,
    mu_x=14.0,
    mu_y=60000.0,
    sigma_x=2.5,
    sigma_y=15000.0,
    rho=0.7,
    break_1=None,
    break_2=None,
    jump_1=0,
    jump_2=0,
    x_min=5.0,
    x_max=20.0,
    seed=None
):
    """
    Monte Carlo study of OLS slope estimator with optional discontinuities.
    Uses exact analytical BLP for plotting.

    Parameters
    ----------
    n, m : int
        Sample size per replication, number of Monte Carlo replications.
    mu_x, mu_y, sigma_x, sigma_y, rho : float
        Parameters of bivariate normal.
    break_1, break_2 : float or None
        Locations of discontinuities.
    jump_1, jump_2 : float
        Magnitude of discontinuities.
    x_min, x_max : float
        Plotting range.
    seed : int or None
        Random seed.
    """

    if seed is not None:
        np.random.seed(seed)

    x_grid = np.linspace(x_min, x_max, 200)

    # ---- Monte Carlo OLS sampling ----

    # Use these arrays to keep track of the slopes and fitted lines. 
    slopes = []
    fitted_lines = []

    for _ in range(m):
        X,y = sample_schooling_income(
            n=n,
            mu_x=mu_x,
            mu_y=mu_y,
            sigma_x=sigma_x,
            sigma_y=sigma_y,
            rho=rho,
            break_1=break_1,
            break_2=break_2,
            jump_1=jump_1,
            jump_2=jump_2,
        )

        # YOUR CODE HERE: OLS fit using statsmodels.api.OLS (don't forget to add the constant and use HC1 as cov type)
        # Should be close to 5 lines

        

    slopes = np.array(slopes)

    
    # ---- Analytical BLP ----
    # Base slope
    beta1_base = rho * sigma_y / sigma_x

    # Contribution from jumps
    phi1 = norm.pdf(0) if break_1 is None else norm.pdf((break_1 - mu_x)/sigma_x)
    phi2 = norm.pdf(0) if break_2 is None else norm.pdf((break_2 - mu_x)/sigma_x)
    beta1_blp = beta1_base + (jump_1 * phi1 + jump_2 * phi2)/sigma_x

    # Beta0
    FX1 = 0.0 if break_1 is None else norm.cdf((break_1 - mu_x)/sigma_x)
    FX2 = 0.0 if break_2 is None else norm.cdf((break_2 - mu_x)/sigma_x)
    beta0_blp = mu_y + jump_1*(1 - FX1) + jump_2*(1 - FX2) - beta1_blp * mu_x

    blp_line = beta0_blp + beta1_blp * x_grid

    # ---- True CMF with discontinuities ----
    true_cmf = mu_y + beta1_base * (x_grid - mu_x)
    if break_1 is not None:
        true_cmf[x_grid >= break_1] += jump_1
    if break_2 is not None:
        true_cmf[x_grid >= break_2] += jump_2

    # ---- Plotting ----
    def thousands_formatter(val, pos):
        return f"{int(val/1000)}k"

    fig, axes = plt.subplots(1,2,figsize=(12,4), dpi=150)

    # Histogram
    axes[0].hist(slopes, bins=40, density=True, color="tab:orange", alpha=0.7)
    axes[0].axvline(beta1_blp, color="green", linestyle="--", lw=2, label="Population BLP slope")
    axes[0].set_xlabel("Estimated Slope")
    axes[0].set_ylabel("Density")
    axes[0].set_title("Sampling Distribution of OLS Slope")
    axes[0].legend()
    axes[0].spines["top"].set_visible(False)
    axes[0].spines["right"].set_visible(False)

    # Regression lines
    for line in fitted_lines:
        axes[1].plot(x_grid, line, color="tab:orange", alpha=0.05)
    axes[1].plot(x_grid, true_cmf, color="tab:blue", lw=2.5, label="True CMF")
    axes[1].plot(x_grid, blp_line, color="green", lw=2, linestyle="--", label="Best Linear Predictor")
    axes[1].set_xlabel("Years of Completed Schooling")
    axes[1].set_ylabel("Annual Income")
    axes[1].set_ylim(20000, 100000)
    axes[1].yaxis.set_major_formatter(FuncFormatter(thousands_formatter))
    axes[1].set_title("OLS Fits with True CMF and BLP")
    axes[1].legend()
    axes[1].spines["top"].set_visible(False)
    axes[1].spines["right"].set_visible(False)

    plt.tight_layout()
    plt.show()

In [ ]:
ols_repeated_sampling(
    n=1000,
    m=1000,
    break_1=12, 
    break_2=16,
    jump_1=5000, 
    jump_2=10000,
)

# Exercise 2

In [ ]:
n = 1000

X_a, y_a = sample_schooling_income(
    n=n,
    mu_x=14.0,
    mu_y=60000.0,
    sigma_x=2.5,
    sigma_y=15000.0,
    rho=0.0,
    break_1=12.0,
    break_2=16.0,
    jump_1=0,
    jump_2=0,
)

X_b, y_b = sample_schooling_income(
    n=n,
    mu_x=14.0,
    mu_y=60000.0,
    sigma_x=2.5,
    sigma_y=15000.0,
    rho=0.7,
    break_1=12.0,
    break_2=16.0,
    jump_1=0,
    jump_2=0,
)

X_c, y_c = sample_schooling_income(
    n=n,
    mu_x=14.0,
    mu_y=60000.0,
    sigma_x=2.5,
    sigma_y=15000.0,
    rho=0.7,
    break_1=12.0,
    break_2=16.0,
    jump_1=5000,
    jump_2=10000,
)

In [ ]:
# YOUR CODE HERE: use statsmodels.api.ols to estimate and display the regression tables for each sample
# Don't forget to add the constant and use HC1 as cov type
# 3-liner per sample